In [ ]:
##===================(PRICE DATA DOWNLOADER)===================##
##  This script downloads price data from yahoo finance for all tickers in a file avalable on the SEC website
## The file is available at https://www.sec.gov/include/ticker.txt
## After downloding all the data it will make indicators and then plot them against the percent chaneg in price 
##==============================================================##
!pip install numpy
!pip install pandas
!pip install scipy
!pip install yfinance
!pip install plotly
!pip install tqdm
!pip install scikit-learn

In [ ]:
import os
import time
import numpy as np
import random
import yfinance as yf
import scipy.stats as stats
from tqdm import tqdm
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from tqdm.notebook import tqdm
from random import uniform
import pandas as pd
import plotly.express as px
from scipy.signal import find_peaks
from scipy.stats import linregress

In [ ]:
##===================(PRICE DATA DOWNLOADER)===================##
##Roughly 11895 tickers with a download time of 1.05 seconds per ticker
##Need to add an updated ticker list that only uses good tickers for week day data downloads and a seperate list for weekend downloads
##Estimated time to run 3:30:00 ~ 3 hours 30 minutes

def read_tickers(file_path):
    with open(file_path, 'r') as file:
        tickers = [line.split('\t')[0] for line in file.readlines()]
    return tickers

def fetch_and_save_data(tickers, output_dir):
    last_call_time = None
    with tqdm(total=len(tickers), desc='Fetching Data') as pbar:
        for ticker in tickers:
            if last_call_time is not None:
                elapsed = time.time() - last_call_time
                if elapsed < 1.05:
                    time.sleep(1.05 - elapsed)
            
            data = yf.download(ticker, interval='1d', period="max", progress=False)
            filename = os.path.join(output_dir, f'{ticker}.csv')
            data.to_csv(filename)
            last_call_time = time.time()
            pbar.update(1)

file_path = 'TickersCIK.txt'
output_dir = 'Data/DailyPriceData'
os.makedirs(output_dir, exist_ok=True)
tickers = read_tickers(file_path)
fetch_and_save_data(tickers, output_dir)
print(f'Completed: {len(os.listdir(output_dir))} out of {len(tickers)}')

In [4]:
##===================(PRICE DATA CLEANER)===================##
##Estimated Time to run 0:00:30 ~ 30 seconds

def clean_price_data(folder_path):
    """
    Cleans stock price data files based on specified criteria and provides diagnostics.

    Args:
    folder_path (str): Path to the folder containing stock price data files.

    Returns:
    None: Files are cleaned, and diagnostics are printed.
    """
    total_files = 0
    removed_files = 0
    filetooshort = 0
    removed_files_mean_close = []

    for file_name in tqdm(os.listdir(folder_path), desc='Cleaning Data'):
        total_files += 1
        file_path = os.path.join(folder_path, file_name)

        # Read data
        try:
            data = pd.read_csv(file_path)
        except Exception as e:
            print(f"Error reading {file_name}: {e}")
            continue

        # Check criteria
        remove_file = False

        # Criteria 1: Remove if any closing price is greater than 100,000
        if data['Close'].max() > 100000:
            remove_file = True

        # Criteria 2: Remove if closing prices are below 0 for more than 5 rows
        if (data['Close'] < 0).sum() > 5:
            remove_file = True

        # Criteria 3: Remove if more than 1% NaN values
        if data.isnull().mean().mean() > 0.01:
            remove_file = True

        # Criteria 4: Remove if less than 500 days of data
        if len(data) < 500:
            remove_file = True
            filetooshort += 1

        ## Criteria 5: Remove if the closing price is less than 0.01 for more than 1/3 of the data
        if (data['Close'] < 0.01).sum() > len(data)/3:
            remove_file = True

        ## Criteria 6: remove if the date does not extend to 2022
        if data['Date'].max() < '2022-01-01':
            remove_file = True
            
        # Criteria 7: Remove if the volume * the closing price is less than 10,000 for more than 1/3 of the data
        if (data['Close'] * data['Volume'] < 10000).sum() > len(data)/3:
            remove_file = True

        # Process file removal
        if remove_file:
            removed_files += 1
            removed_files_mean_close.append(data['Close'].mean())
            os.remove(file_path)

    # Display diagnostics
    if total_files > 0:
        print(f"Total files processed: {total_files}")
        print(f"Total files removed: {removed_files}")
        print(f"Percentage of files removed: {removed_files / total_files * 100:.2f}%")
        if removed_files > 0:
            print(f"Mean closing price of removed files: {sum(removed_files_mean_close) / removed_files:.2f}")
    else:
        print("No files to process.")

# Use the function
clean_price_data('Data\DailyPriceData')

Cleaning Data:   0%|          | 0/8236 [00:00<?, ?it/s]

Total files processed: 8236
Total files removed: 2339
Percentage of files removed: 28.40%
Mean closing price of removed files: 76.09


In [48]:
##========================(INDICATOR CALCULATOR)========================##
#Estimated time to run 00:45:00 ~ 45 minutes
#Target time to run 0:10:00 ~ 10 minutes

def calculate_slope(series):
    if series.isnull().all() or len(series.dropna()) <= 1:
        return np.nan
    else:
        return stats.linregress(range(len(series)), series.values)[0]

def indicators(df):
    close = df['Close']
    high = df['High']
    low = df['Low']
    volume = df['Volume']
    close_shift_1 = close.shift(1)
    true_range = np.maximum(high - low, np.maximum(np.abs(high - close_shift_1), np.abs(close_shift_1 - low)))
    df['ATR'] = true_range.rolling(window=14).mean() 
    df['ATR%'] = (df['ATR'] / close) * 100
    df['200ma'] = close.rolling(window=200).mean()
    df['14ma'] = close.rolling(window=14).mean()
    df['14ma%'] = ((close - df['14ma']) / df['14ma']) * 100
    df['200ma%'] = ((close - df['200ma']) / df['200ma']) * 100
    df['14ma-200ma'] = df['14ma'] - df['200ma']
    df['14ma%_change'] = df['14ma%'].pct_change()
    df['14ma%_count'] = df['14ma%'].rolling(window=14).apply(lambda x: np.sum(x > 0))
    df['200ma%_count'] = df['200ma%'].rolling(window=200).apply(lambda x: np.sum(x > 0))
    df['14ma_crossover'] = (close > df['14ma']).astype(int)
    df['200ma_crossover'] = (close > df['200ma']).astype(int)
    df['200DAY_ATR'] = df['200ma'] + df['ATR']
    df['200DAY_ATR-'] = df['200ma'] - df['ATR']
    df['200DAY_ATR%'] = df['200DAY_ATR'] / close
    df['200DAY_ATR-%'] = df['200DAY_ATR-'] / close
    df['percent_from_high'] = ((close - close.cummax()) / close.cummax()) * 100
    df['new_high'] = (close == close.cummax())
    df['days_since_high'] = (~df['new_high']).cumsum() - (~df['new_high']).cumsum().where(df['new_high']).ffill().fillna(0)
    df['percent_range'] = (high - low) / close * 100
    typical_price = (high + low + close) / 3
    df['VWAP'] = (typical_price * volume).rolling(window=14).sum() / volume.rolling(window=14).sum()
    df['VWAP_std14'] = df['VWAP'].rolling(window=14).std()
    df['VWAP_std200'] = df['VWAP'].rolling(window=20).std()
    df['VWAP%'] = ((close - df['VWAP']) / df['VWAP']) * 100
    df['VWAP%_from_high'] = ((df['VWAP'] - close.cummax()) / close.cummax()) * 100
    obv_condition = df['Close'] > close_shift_1
    df['OBV'] = np.where(obv_condition, volume, np.where(~obv_condition, -volume, 0)).cumsum()
    df['OBV_change'] = df['OBV'].diff()
    df['Close_change'] = close.diff()
    df['OBV_change%'] = df['OBV_change'] / df['OBV']
    df['OBV_change%_std'] = df['OBV_change%'].rolling(window=14).std()
    df['OBV_change%_std>1'] = (df['OBV_change%_std'] > 1).astype(int)
    df['OBV_change_std'] = df['OBV_change'].rolling(window=14).std()
    df['OBV_change_std>1'] = (df['OBV_change_std'] > 1).astype(int)
    df['OBV_Divergence'] = (np.sign(df['OBV_change']) != np.sign(df['Close_change'])) & (df['OBV_change'].abs() > df['OBV'].rolling(window=14).std())
    df = df.round(8)
    df = calculate_complex_indicators(df)
    df = df.round(8)
    return df


def MinMaxSlope(df, window_size=720):
    df = df.reset_index(drop=True)
    local_min = df['Close'].rolling(window=window_size, min_periods=1).min()
    local_max = df['Close'].rolling(window=window_size, min_periods=1).max()
    min_indices = df['Close'] == local_min
    max_indices = df['Close'] == local_max
    df['days_since_local_min'] = (~min_indices).cumsum()
    df['days_since_local_max'] = (~max_indices).cumsum()
    df['days_since_local_min'] = df['days_since_local_min'] - df['days_since_local_min'][min_indices].reindex(df.index, method='ffill').fillna(0)
    df['days_since_local_max'] = df['days_since_local_max'] - df['days_since_local_max'][max_indices].reindex(df.index, method='ffill').fillna(0)
    df['slope_min_max'] = np.nan

    for idx in np.where(min_indices & (df['days_since_local_max'] <= window_size))[0]:
        if idx - int(df.at[idx, 'days_since_local_max']) >= 0:
            min_idx = idx - int(df.at[idx, 'days_since_local_max'])
            max_idx = idx
            delta_y = df.at[max_idx, 'Close'] - df.at[min_idx, 'Close']
            delta_x = max_idx - min_idx
            df.at[min_idx, 'slope_min_max'] = delta_y / delta_x if delta_x != 0 else np.nan

    return df

def find_local_extrema(df, column='Close', window_size=90):
    """
    Find local maxima and minima (extrema) over a rolling window.

    Args:
    df (DataFrame): DataFrame with stock data.
    column (str): Column name to find extrema.
    window_size (int): Window size for rolling calculation.

    Returns:
    DataFrame: DataFrame with local maxima and minima indices.
    """
    local_maxima = find_peaks(df[column], distance=window_size)[0]
    local_minima = find_peaks(-df[column], distance=window_size)[0]
    df['Local_Max'] = 0
    df['Local_Min'] = 0
    df.loc[local_maxima, 'Local_Max'] = 1
    df.loc[local_minima, 'Local_Min'] = 1

    return df

def best_fit_lines(df, column='Close'):
    """
    Calculate best fit lines through local extrema.

    Args:
    df (DataFrame): DataFrame with local extrema information.

    Returns:
    DataFrame: DataFrame with slope and intercept of best fit lines.
    """
    max_indices = df.index[df['Local_Max'] == 1]
    max_values = df.loc[max_indices, column]
    slope_max, intercept_max, _, _, _ = linregress(max_indices, max_values)
    min_indices = df.index[df['Local_Min'] == 1]
    min_values = df.loc[min_indices, column]
    slope_min, intercept_min, _, _, _ = linregress(min_indices, min_values)
    return slope_max, intercept_max, slope_min, intercept_min

def calculate_channel_metrics(df, slope_max, intercept_max, slope_min, intercept_min, column='Close'):
    """
    Calculate channel metrics and detect channel breakouts.

    Args:
    df (DataFrame): DataFrame with stock data.
    slope_max (float): Slope of the best fit line for local maxima.
    intercept_max (float): Intercept of the best fit line for local maxima.
    slope_min (float): Slope of the best fit line for local minima.
    intercept_min (float): Intercept of the best fit line for local minima.
    column (str): Column name to use for breakout detection.

    Returns:
    DataFrame: Updated DataFrame with channel metrics and breakout information.
    """
    # Calculate channel lines
    df['Top_Channel_Line'] = slope_max * df.index + intercept_max
    df['Bottom_Channel_Line'] = slope_min * df.index + intercept_min
    df['In_Channel'] = ((df[column] <= df['Top_Channel_Line']) & (df[column] >= df['Bottom_Channel_Line'])).astype(int)
    df['Channel_Duration'] = df['In_Channel'].cumsum()
    df['Average_Slope'] = (slope_max + slope_min) / 2
    df['Breakout_Above'] = (df[column] > df['Top_Channel_Line']).astype(int)
    df['Breakout_Below'] = (df[column] < df['Bottom_Channel_Line']).astype(int)
    df['Breakout_Magnitude'] = np.where(df['Breakout_Above'], df[column] - df['Top_Channel_Line'], 
        np.where(df['Breakout_Below'], df['Bottom_Channel_Line'] - df[column], 0))

    return df



def calculate_complex_indicators(df):
    timer = time.time()
    df = MinMaxSlope(df)
    
    close = df['Close']
    high = df['High']
    low = df['Low']

    close_shift_1 = close.shift(1)
    low_shift_1 = low.shift(1)
    high_shift_1 = high.shift(1)

    df['percent_change_Close'] = df['Close'].pct_change()
    df['pct_change_std'] = df['percent_change_Close'].rolling(window=20).std()
    for shift in range(1, 4):
        df[f'percent_change_Close_shift_{shift}'] = df['percent_change_Close'].shift(shift)

    df['percent_change_Close_7'] = df['Close'].pct_change(periods=7)
    df['percent_change_Close_30'] = df['Close'].pct_change(periods=30)
    df['percent_change_Close>std'] = (df['percent_change_Close'] > df['pct_change_std']).astype(int)
    df['percent_change_Close<std'] = (df['percent_change_Close'] < df['pct_change_std']).astype(int)
    df['pct_change_std_rolling'] = df['pct_change_std'].rolling(window=14).mean()
    df['pct_change_std_rolling'] = df['pct_change_std'].rolling(window=60).mean()

    df['percent_change_Close>0'] = (df['percent_change_Close'] > 0).astype(int)
    df['percent_change_Close<0'] = (df['percent_change_Close'] < 0).astype(int)
    df['percent_change_Close>0_count'] = df['percent_change_Close>0'].rolling(window=14).sum()
    df['percent_change_Close<0_count'] = df['percent_change_Close<0'].rolling(window=14).sum()
    df['abs_pct_change_Close_7'] = df['percent_change_Close'].abs().rolling(window=7).mean()
    df['abs_pct_change_Close_30'] = df['percent_change_Close'].abs().rolling(window=30).mean()
    df['abs_pct_change_Close_7>abs_pct_change_Close_30'] = (df['abs_pct_change_Close_7'] > df['abs_pct_change_Close_30']).astype(int)
    df['abs_pct_change_Close_7<abs_pct_change_Close_30'] = (df['abs_pct_change_Close_7'] < df['abs_pct_change_Close_30']).astype(int)

    pct_change_close = df['percent_change_Close']
    close_shift_1 = df['Close'].shift(1)
    low_shift_1 = df['Low'].shift(1)
    high_shift_1 = df['High'].shift(1)
    threshold_multiplier = 0.65
    abnormal_pct_change_threshold = pct_change_close.rolling(window=20).mean() + threshold_multiplier * df['pct_change_std']
    df['days_since_abnormal_pct_change'] = (pct_change_close > abnormal_pct_change_threshold).cumsum()
    significant_figures = [10, 100, 500, 1000]
    custom_values = [50,69,420,666,777,888,999]
    multiples = [x for figure in significant_figures for x in range(figure, int(df['Close'].max()) + 1, figure)]
    psych_levels = set(multiples + custom_values)
    rolling_min = df['Close'].rolling(window=500).min()
    df['psych_level'] = df['Close'].apply(lambda x: any(abs(x - level) / level <= 0.01 for level in psych_levels))
    df['psych_level'] |= (abs(df['Close'] - rolling_min) / rolling_min <= 0.01)
    df['psych_level%'] = (df['Close'] - rolling_min) / rolling_min * 100
    df['days_since_psych_level'] = (~df['psych_level']).cumsum() - (~df['psych_level']).cumsum().where(df['psych_level']).ffill().fillna(0)
    df['direction_flipper'] = (pct_change_close > 0).astype(int)
    df['direction_flipper_count5'] = df['direction_flipper'].rolling(window=5).sum()
    df['direction_flipper_count_10'] = df['direction_flipper'].rolling(window=10).sum()
    df['direction_flipper_count_14'] = df['direction_flipper'].rolling(window=14).sum()
    df['ATR'] = df['Close'].rolling(window=14).apply(lambda x: np.mean(np.abs(np.diff(x))))
    keltner_central = df['Close'].ewm(span=20).mean()
    keltner_range = df['ATR'] * 1.5
    df['KC_UPPER%'] = ((keltner_central + keltner_range) - df['Close']) / df['Close'] * 100
    df['KC_LOWER%'] = (df['Close'] - (keltner_central - keltner_range)) / df['Close'] * 100
    delta = df['Close'].diff()
    gain = delta.clip(lower=0).rolling(window=14).mean()
    loss = (-delta).clip(lower=0).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    rolling_mean = df['Close'].rolling(window=20).mean()
    rolling_std = df['Close'].rolling(window=20).std()
    df['B%'] = (df['Close'] - (rolling_mean - 2 * rolling_std)) / (4 * rolling_std)
    positive_streak = (pct_change_close > 0).astype(int).cumsum()
    negative_streak = (pct_change_close < 0).astype(int).cumsum()
    df['positive_streak'] = positive_streak - positive_streak.where(pct_change_close <= 0).ffill().fillna(0)
    df['negative_streak'] = negative_streak - negative_streak.where(pct_change_close >= 0).ffill().fillna(0)
    gap_threshold_percent = 0.5 if pct_change_close.std() > 1 else 0
    df['is_gap_up'] = (df['Low'] - high_shift_1) / close_shift_1 * 100 > gap_threshold_percent
    df['is_gap_down'] = (df['High'] - low_shift_1) / close_shift_1 * 100 < -gap_threshold_percent
    df['move_from_gap%'] = np.where(df['is_gap_up'], (df['Close'] - low_shift_1) / low_shift_1 * 100,
        np.where(df['is_gap_down'], (df['Close'] - high_shift_1) / high_shift_1 * 100, 0))
    df['VPT'] = df['Volume'] * ((df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1))




    df = find_local_extrema(df, column='Close', window_size=20)
    slope_max, intercept_max, slope_min, intercept_min = best_fit_lines(df)
    df = calculate_channel_metrics(df, slope_max, intercept_max, slope_min, intercept_min, column='Close')
    df = df.drop(['Open', 'High', 'Low', 'Close', 'Adj Close', 'VWAP', '200DAY_ATR-', '200DAY_ATR', 'Close_change', 'Volume', 'ATR', 'OBV', '200ma', '14ma'], axis=1)
    timer2 = time.time()
    print(f"Time to run complex indicator function: {round(timer2 - timer, 3)} seconds")
    return df






##===================(CONTROL LOGIC)===================##
##===================(CONTROL LOGIC)===================##
##===================(CONTROL LOGIC)===================##


def IntermediaiteDataProcessing(df):
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.replace([True, False], [1, 0])
    df = df.bfill()
    df = df.ffill()    
    df = df.iloc[int(len(df) * 0.1):]    
    return df


def squash_col_outliers(df, col_name=None, num_std_dev=3):
    columns_to_process = [col_name] if col_name is not None else df.select_dtypes(include=['float64']).columns
    for col in columns_to_process:
        if col not in df.columns or df[col].dtype != 'float64':
            continue  # Skip non-existent or non-float columns
        mean = df[df[col] != 0][col].mean()
        std_dev = df[df[col] != 0][col].std()
        lower_bound = mean - num_std_dev * std_dev
        upper_bound = mean + num_std_dev * std_dev
        num_lower_clipped = (df[col] < lower_bound).sum()
        num_upper_clipped = (df[col] > upper_bound).sum()
        df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)
        print(f"Column '{col}': Clipped {num_lower_clipped} lower outliers and {num_upper_clipped} upper outliers.")

    return df


def scale_column(col, precision_type='float32'):
    if col.dtype == 'bool':
        return col
    if col.mean() > 3:
        col = col.apply(lambda x: x + uniform(-0.001, 0.001) if x not in [0, 1] else x)
    
    scaler = MinMaxScaler() if col.dtype in ['int64', 'int32'] else RobustScaler()
    scaled_col = scaler.fit_transform(col.values.reshape(-1, 1))
    return scaled_col.astype(precision_type)

def scale_data(df, precision_type='float32'):
    df = df.copy()
    df.ffill(inplace=True)
    df.bfill(inplace=True)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    scalers = {}

    for col_name in df.select_dtypes(include=['number']).columns:
        if df[col_name].nunique() == 2 and sorted(df[col_name].unique()) == [0, 1]:
            continue
        scaled_col = scale_column(df[col_name], precision_type)
        df[col_name] = scaled_col
        if df[col_name].sum() == 0:
            continue
        scaler = MinMaxScaler() if df[col_name].dtype in ['int64', 'int32'] else RobustScaler()
        scaler.fit(df[col_name].values.reshape(-1, 1))
        scalers[f'scaler_{col_name}'] = scaler
    return df, scalers


def process_stock_data(file_path, output_dir, Testing):
    """
    Reads a CSV file, computes indicators, scales the data, and saves it to a new directory.
    Skips files with fewer than `min_rows` rows of data.
    """
    df = pd.read_csv(file_path)
    df = df.tail(4000)
    df = indicators(df)
    if Testing == False:
        df, _ = scale_data(df)
    df = IntermediaiteDataProcessing(df)
    df = squash_col_outliers(df, col_name='specific_column', num_std_dev=2)
    output_file = os.path.join(output_dir, os.path.basename(file_path))
    df.to_csv(output_file, index=False)



def process_files(input_dir, output_dir, Testing):
    """
    Process all stock data files in the specified input directory and save them to the output directory.
    """
    os.makedirs(output_dir, exist_ok=True)  # Create output directory if it doesn't exist
    files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]
    if Testing == True:
        files = files[:50]
    for file in tqdm(files, desc="Processing Files"):
        file_path = os.path.join(input_dir, file)
        process_stock_data(file_path, output_dir,Testing)



def main(Testing):
    input_dir = 'Data/DailyPriceData'
    output_dir = 'Data/ProcessedPriceData'
    process_files(input_dir, output_dir, Testing)



if __name__ == "__main__":
    
    folder = 'Data/ProcessedPriceData'
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)

        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))


    Testing = True
    main(Testing)

Processing Files:   0%|          | 0/50 [00:00<?, ?it/s]

Time to run complex indicator function: 0.07 seconds
Time to run complex indicator function: 0.07 seconds
Time to run complex indicator function: 0.03 seconds
Time to run complex indicator function: 0.07 seconds
Time to run complex indicator function: 0.04 seconds
Time to run complex indicator function: 0.02 seconds
Time to run complex indicator function: 0.03 seconds
Time to run complex indicator function: 0.07 seconds
Time to run complex indicator function: 0.02 seconds
Time to run complex indicator function: 0.07 seconds
Time to run complex indicator function: 0.02 seconds
Time to run complex indicator function: 0.05 seconds
Time to run complex indicator function: 0.07 seconds
Time to run complex indicator function: 0.08 seconds
Time to run complex indicator function: 0.07 seconds
Time to run complex indicator function: 0.06 seconds
Time to run complex indicator function: 0.07 seconds
Time to run complex indicator function: 0.06 seconds
Time to run complex indicator function: 0.07 s

In [ ]:
##===================(Data Testing/Improvment)===================##
##if testing is true then only 50 files will be processed Estimated time to run 0:00:01 ~ 1 seconds
##if testing is false then all files will be processed Estimated time to run 0:00:10 ~ 10 seconds

columns_of_interest = ['percent_change_Close'] 
folder_path = 'Data\ProcessedPriceData'  
num_files_to_sample = 25
output_csv_folder = 'Data\Logging'

def select_random_files(folder, num_files):
    all_files = [f for f in os.listdir(folder) if f.endswith('.csv')]
    return random.sample(all_files, min(num_files, len(all_files)))

def calculate_correlations_for_column(df, column):
    if column in df.columns:
        if df[column].sum() == 0:
            return None
        numeric_df = df.select_dtypes(include=[np.number])  # Select only numeric columns
        if len(numeric_df) > 0:
            return numeric_df.corrwith(numeric_df[column]).drop(column)
    return None

def process_files_and_save(folder, files, columns, output_folder):
    correlations = {col: pd.Series(dtype='float') for col in columns}
    for file in files:
        df = pd.read_csv(os.path.join(folder, file))
        for col in columns:
            corr_series = calculate_correlations_for_column(df, col)
            if corr_series is not None:
                correlations[col] = correlations[col].add(corr_series, fill_value=0)

    for col, corr_series in correlations.items():
        avg_corr = corr_series / len(files)
        sorted_corr = avg_corr.sort_values(ascending=False).reset_index()
        sorted_corr.columns = ['Column', 'Correlation']
        sorted_corr.to_csv(os.path.join(output_folder, f'correlation_with_{col}.csv'), index=False)
    plot_correlation(avg_corr, col)

def plot_correlation(correlation_series, column_name):
    sorted_corr = correlation_series.sort_values(ascending=False)

    fig = px.bar(sorted_corr, 
                 labels={'index': 'Column', 'value': 'Correlation'},
                 title=f'Average Correlation with {column_name}')
    fig.update_layout(xaxis_title='Column', yaxis_title='Correlation')
    fig.show()

if len(os.listdir(folder_path)) > 100:
    num_files_to_sample = int(len(os.listdir(folder_path)) / 3)

selected_files = select_random_files(folder_path, num_files_to_sample)
process_files_and_save(folder_path, selected_files, columns_of_interest, output_csv_folder)